In [1]:
!pip install pandahouse
!pip install swifter

  Preparing metadata (setup.py) ... done
  Created wheel for pandahouse: filename=pandahouse-0.2.7-py2.py3-none-any.whl size=5904 sha256=945fbd377e14c07777fadd2c9ee6232bffc3aa02e3ca1d2cc501a2a96867895e
  Stored in directory: /root/.cache/pip/wheels/04/81/63/6896730711da10812121737bb505c6a8993800e99d39432522
Successfully built pandahouse
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for swifter: filename=swifter-1.4.0-py3-none-any.whl size=16507 sha256=4e82acfaa503a3eb694c010bfd3b9b9747746d3b1799b257e95fa73a497a5807
  Stored in directory: /root/.cache/pip/wheels/e4/cf/51/0904952972ee2c7aa3709437065278dc534ec1b8d2ad41b443
Successfully built swifter


In [1]:
import pandas as pd
import numpy as np
import pandahouse
import swifter
import seaborn as sns

# Сравнить retention по источникам траффика
Математически самым простым вариантом видится расчёт retention с помощью churn:

$retention = 1-churn\_rate$

$churn\_rate = \frac{Ушедшие \; пользователи}{Все \; пользователи}$

При этом возникает вопрос кого считать ушедшим пользоватлем:
- пользователя, который последний раз заходил $n$ дней назад

### Определение приемлемого времени для записи пользователя в отток

In [25]:
connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20240320'
}

In [26]:
q = """
WITH
  t_user_activity_dates AS (
    -- Дни, когда активничал пользователь
    SELECT DISTINCT
      user_id,
      DATE(time) AS date
    FROM {db}.feed_actions
  ),
  t_last_activity AS (
    -- Уникальные пользователи
    SELECT
      user_id,
      MAX(date) AS last_date
    FROM t_user_activity_dates
    GROUP BY user_id
  )

SELECT *
FROM t_user_activity_dates
ORDER BY user_id, date
"""

df = pandahouse.read_clickhouse(q, connection=connection)
df.head(50)

,user_id,date
0,200,2024-02-03
1,200,2024-02-05
2,200,2024-02-08
3,200,2024-02-10
4,200,2024-02-13
5,200,2024-03-04
6,200,2024-03-14
7,200,2024-03-18
8,200,2024-03-22
9,201,2024-02-03
